# Transfer Learning in Keras

In this notebook, we'll cover how to load a pre-trained model (in this case, VGGNet19) and finetune it for a new task: detecting hot dogs.

#### Load dependencies

In [1]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

2023-07-15 23:04:08.246230: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Load the pre-trained VGG19 model

In [2]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

2023-07-15 23:04:22.911241: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-15 23:04:23.158355: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-15 23:04:23.158402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-15 23:04:23.173136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-15 23:04:23.173183: I tensorflow/compile

80134624/80134624 [==============================] - 6s 0us/step


#### Freeze all the layers in the base VGGNet19 model

In [3]:
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

In [4]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

#### Compile the model for training

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

The dataset is available for download [here](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home). You should download the zipfile and extract the contents into a folder called `hot-dog-not-hot-dog` in the `notebooks` directory.

In [6]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [7]:
# Define the batch size:
batch_size=32

In [11]:
# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='../data/hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='../data/hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [12]:
model.fit_generator(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15)

/tmp/ipykernel_227/2169329564.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=15,


Epoch 1/16


2023-07-15 23:07:48.321198: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-15 23:07:50.149520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-07-15 23:07:52.183436: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.72GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-15 23:07:52.731272: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.45GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be

 1/15 [=>............................] - ETA: 2:48 - loss: 1.4159 - accuracy: 0.4062

2023-07-15 23:08:00.341834: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


15/15 [==============================] - ETA: 0s - loss: 1.3694 - accuracy: 0.5193

2023-07-15 23:08:21.510007: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


15/15 [==============================] - 39s 2s/step - loss: 1.3694 - accuracy: 0.5193 - val_loss: 0.5197 - val_accuracy: 0.7604
Epoch 2/16
15/15 [==============================] - 12s 801ms/step - loss: 0.6579 - accuracy: 0.6931 - val_loss: 0.5975 - val_accuracy: 0.7208
Epoch 3/16
15/15 [==============================] - 10s 685ms/step - loss: 0.6111 - accuracy: 0.7210 - val_loss: 0.6444 - val_accuracy: 0.7063
Epoch 4/16
15/15 [==============================] - 10s 707ms/step - loss: 0.5147 - accuracy: 0.7876 - val_loss: 0.4615 - val_accuracy: 0.7646
Epoch 5/16
15/15 [==============================] - 10s 676ms/step - loss: 0.4189 - accuracy: 0.8112 - val_loss: 0.4623 - val_accuracy: 0.7708
Epoch 6/16
15/15 [==============================] - 10s 681ms/step - loss: 0.3832 - accuracy: 0.8262 - val_loss: 0.4555 - val_accuracy: 0.7708
Epoch 7/16
15/15 [==============================] - 10s 676ms/step - loss: 0.3367 - accuracy: 0.8605 - val_loss: 0.4515 - val_accuracy: 0.7708
Epoch 8/16
15